**The following tutorial demonstrates how to use reDA for identifying cell states associated with phenotype in a scATAC-seq dataset. And we provide a simulated dataset for testing.**

# Software requirements

Before using, we need to make sure that the following packages have been installed in the system

Python Dependencies

In [ ]:
numpy 1.26.2
scipy 1.11.3
pandas 1.3.5
argparse 1.4.0
scanpy 1.9.5
packaging 23.2
anndata 0.10.2
rpy2 3.4.5
multianndata 0.0.4

R Dependencies

In [ ]:
Signac 1.12.9001
Seurat 5.0.1
harmony 0.1.1
readr 2.1.5
GenomicRanges 1.50.2
SeuratDisk 0.0.0.9020

# How to install

reDA is developed under Python (version >= 3.9) and R (version >= 4.2). To use reDA, you can clone this repository:

In [ ]:
git clone https://github.com/Jinsl-lab/reDA.git 
cd reDA

Then install the reDA package.

In [ ]:
pip install .

# Start in python

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import reDA
import anndata as ad
from multianndata import MultiAnnData

# 1. pre-processing

Firstly, we preprocess the scATAC-seq data in the .rds format and save it as the .h5ad format available for python.

In [ ]:
reDA.pp.preprocess(rpath="/....../reDA/rpyprc.R",  #The path of rpyprc.R
                   inpath="/....../scATAC_data.rds",  #Input scATAC-seq raw data
                   hadpath="/....../output_data.h5Seurat", #The save path of the output data
                   group="dataset", #Features used to remove batch effects
                   assay="ATAC",
                   dimsta=2,dimend=30,  #The dimension used to calculate the cell similarity matrix in the dimension reduction matrix
                   issave="Yes", #Whether to save the preprocessed data in .rds format
                   rdspath="/....../output_data.rds") #The save path of the .rds data

# 2. read in data

Then, we create a MultiAnnData object from an AnnData object.

In [ ]:
# read in AnnData object from h5ad file 
d = ad.read_h5ad('/....../output_data.h5ad')
d.obs["id"]=d.obs["id"].astype('int')
d = MultiAnnData(d, sampleid='id') 
d.obs_to_sample(['Diagnosis'])

# 3. perform association test 

In [ ]:
res = reDA.tl.association(d,  #dataset                 
            d.samplem.Diagnosis,  #phenotypes of interest
            re=0.3)   #restart probability                       
            #covs=d.samplem[['gender']]) #covariates(optional)

1. **ncorrs** contains neighborhood coefficients.  
2. **fdrs** contains neighborhood-level FDRs at other thresholds.


In [ ]:
passed = res.fdrs[res.fdrs.fdr <= 0.05]
thresh = passed.threshold.iloc[0]
print(thresh)

pre=np.repeat("NotDA", len(d))
pre[res.ncorrs > thresh]="PosLFC"
pre[res.ncorrs < -thresh]="NegLFC"
d.obs["pred"]=pre



In [ ]:
#Show results
import seaborn as sns
colors=sns.color_palette('Set1')
pal=colors[0:3]
pal.insert(1, colors[-1])
sc.pl.umap(d, color='pred', palette=pal)